In [25]:
import pandas as pd
import requests
alphavantage_key = "7QD6TN4TDCX63BZ6"
ticker = "aapl"
months = {"January":"01", "February": "02", "March":"03", "April": "04", "May": "05",
 "June": "06", "July": "07", "August": "08", "September": "09", "October": "10",
 "November": "11", "December": "12"}

In [29]:
def get_ratios(tickers):
    # Get dividend yield, payout ratio and EPS next 1Y from Overview tab on Alphavantage API
    url_overview = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol=' + ticker + '&apikey=' + alphavantage_key
    overview = requests.get(url_overview).json()
    
    #Divident Yield:
    Dividend_yield = round(float(overview['DividendYield'])*100,2)
    print("Dividend Yield: "+ str(Dividend_yield)+ "%")
    
    #Payout ratio:
    Payout_ratio = round(float(overview['DividendPerShare'])/float(overview['EPS'])*100,2)
    print("Payout Ratio: " + str(Payout_ratio)+ "%")
    
    #EPS next 1Y (we get EPS next 1Y from Forward PE formula which is current price/1Y EPS):
    EPS_next_1Y = round(float(overview['ForwardPE'])/(float(overview['EPS'])*float(overview['PERatio']))*100,2)
    print("EPS next 1Y: "+ str(EPS_next_1Y) + "%")
    
    # Get EPS past 5Y from alphavantage API on tab earnings we need the if function because 
    #the first result shown in earnings is only annual if the last quarter is the same as the annual results quarter
    url_earnings = 'https://www.alphavantage.co/query?function=EARNINGS&symbol=' + ticker + '&apikey=' + alphavantage_key
    earnings = requests.get(url_earnings).json()
    if earnings['annualEarnings'][0]['fiscalDateEnding'][5:7] == months[overview['FiscalYearEnd']]:
        last_EPS = float(earnings['annualEarnings'][0]['reportedEPS'])
        five_years_EPS = float(earnings['annualEarnings'][4]['reportedEPS'])
    else:
        last_EPS = float(earnings['annualEarnings'][1]['reportedEPS'])
        five_years_EPS = float(earnings['annualEarnings'][5]['reportedEPS'])
    if last_EPS/five_years_EPS>0:
        EPS_past_5Y = round((((last_EPS/five_years_EPS)**0.2)-1)*100,2)
    elif last_EPS<0:
        EPS_past_5Y =round((((abs(last_EPS/five_years_EPS)**0.2))-1)*-100,2)
    else: EPS_past_5Y =round((((abs(last_EPS/five_years_EPS)**0.2))-1)*100,2)
    print("EPS past 5Y: "+ str(EPS_past_5Y) +"%" )
    
    #Get Debt/FCF from Alphavantage API on tabs CF statement and BS statement
    url_cf = 'https://www.alphavantage.co/query?function=CASH_FLOW&symbol=' + ticker + '&apikey=' + alphavantage_key
    cf = requests.get(url_cf).json()
    url_balance = 'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol=' + ticker + '&apikey=' + alphavantage_key
    balance = requests.get(url_balance).json()
    Debt = float(balance['annualReports'][0]['longTermDebt'])+float(balance['annualReports'][0]['currentDebt'])
    Operating_cf = float(cf['annualReports'][0]['operatingCashflow'])
    Debt_over_FCF = round(Debt/Operating_cf,2)
    print("Debt/FCF: "+ str(Debt_over_FCF) +" years" )
    
    #Return on Equity
    url_income = 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol=' + ticker + '&apikey=' + alphavantage_key
    income = requests.get(url_income).json()
    current_net_income = float(income['annualReports'][0]['netIncome'])
    five_years_old_net_income = float(income['annualReports'][4]['netIncome'])
    current_equity = float(balance['annualReports'][0]['commonStock'])
    five_years_old_equity = float(balance['annualReports'][4]['commonStock'])
    return_on_equity_past_five_years = round(((current_net_income/current_equity)/(five_years_old_net_income/five_years_old_equity)-1),4)*100
    print("ROE past 5Y: "+ str(return_on_equity_past_five_years) +"%")

    return


In [30]:
get_ratios('mmm')

Dividend Yield: 0.6%
Payout Ratio: 15.4%
EPS next 1Y: 14.82%
EPS past 5Y: 15.52%
Debt/FCF: 1.08 years
ROE past 5Y: 3.93%
